In [1]:
#NN Surrogate model class
import injector_surrogate_quads
from injector_surrogate_quads import *
import physics_gp

sys.path.append('../configs')
#Sim reference point to optimize around
from ref_config import ref_point

#Pytorch 
import numpy as np
import torch
import gpytorch
import botorch 

# BO for Minimizing Emittance with 3 Variables (SQ, CQ, SOL)

In [2]:
#load injector model
Model = Surrogate_NN()

Model.load_saved_model(model_path = '../models/', \
                       model_name = 'model_OTR2_NA_rms_emit_elu_2021-07-27T19_54_57-07_00')
Model.load_scaling()
Model.take_log_out = False

2022-06-15 14:44:18.913367: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Objective Function

In [3]:
#convert to machine units
ref_point = Model.sim_to_machine(np.asarray(ref_point))

#input params: solenoid and quads to vary 
opt_var_names = ['SOL1:solenoid_field_scale','SQ01:b1_gradient','CQ01:b1_gradient']
#input bounds
bounds = torch.tensor([[0.44, 0.55],[-0.02, 0.02], [-0.02, 0.02]])

#output params: emittance in transverse plane (x & y)
opt_out_names = ['norm_emit_x','norm_emit_y']

def evaluate(config): 
    """
    D is input space dimensionality
    :param config: input values of opt_var_names, torch.tensor, shape (1, D) 
    """
    #make input array of length model_in_list (inputs model takes)
    x_in = np.empty((1,len(Model.model_in_list)))

    #fill in reference point around which to optimize
    x_in[:,:] = np.asarray(ref_point[0])

    #set solenoid, SQ, CQ to values from optimization step
    for i in range(config.size(dim=0)):
        x_in[:, Model.loc_in[opt_var_names[i]]] = config[i]

    #output predictions
    y_out = Model.pred_machine_units(x_in) 

    return -1*objective(y_out)[0]


def objective(y_out):
    
    #output is geometric emittance in transverse plane
    out1 = y_out[:,Model.loc_out['norm_emit_x']] #grab norm_emit_x out of the model
    out2 = y_out[:,Model.loc_out['norm_emit_y']] #grab norm_emit_y out of the model
       
    return np.sqrt(out1*out2)/1e-6 # in um units

## Gaussian Regression & Acquisition Function

In [4]:
def get_BO_point(x, f, bounds, beta=2.5, input_transform=None, outcome_transform=None, precision = None, phys=True):
    """
    function that trains a GP model of data and returns the next observation point using UCB
    D is input space dimensionality
    N is number of samples

    :param x: input points data, torch.tensor, shape (N,D)
    :param f: output point data, torch.tensor, shape (N,1)
    :param bounds: input space bounds, torch.tensor, shape (2,D)
    :param precision: precision matrix used for RBF kernel (must be PSD), torch.tensor, (D,D)
    :param beta: UCB optimization parameter, float
    :return x_candidate, model: next observation point and gp model w/observations
    """
    
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    
    if phys == True:
        gp = physics_gp.PhysicsExactGPModel(x, f.flatten(), likelihood, precision)
    else:
        gp = botorch.models.SingleTaskGP(x, f, likelihood=likelihood, 
                                         outcome_transform=outcome_transform, 
                                         input_transform=input_transform)
        
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(gp.likelihood, gp)
    
    # fit GP hyperparameters
    botorch.fit.fit_gpytorch_model(mll)

    # do UCB acquisition
    UCB = botorch.acquisition.UpperConfidenceBound(gp, beta=beta)
    candidate, acq_value = botorch.optim.optimize_acqf(UCB,
                                                       bounds=bounds,
                                                       q=1,
                                                       num_restarts=20,#5
                                                       raw_samples=20)
    return candidate, gp

## Set up initial training samples

In [5]:
#create initial samples within specified bounds
n_samples = 5
n_var = 3

train_x = torch.zeros((n_samples, n_var)) 
for i in range(n_var):
    train_x[:,i] = torch.as_tensor(np.random.uniform(bounds[i,0],bounds[i,1],(n_samples,)))
print(train_x)

train_y = torch.tensor([evaluate(vars) for vars in train_x]).reshape(-1,1)
print(train_y)

# transformer 
transformer_x = botorch.models.transforms.input.Normalize(n_var, bounds = bounds.transpose(0,1))
transformer_y = botorch.models.transforms.outcome.Standardize(1)

normed_bounds = torch.cat((torch.zeros(1,n_var), torch.ones(1,n_var)), 0)

tensor([[ 0.5114, -0.0191, -0.0034],
        [ 0.4512,  0.0083,  0.0116],
        [ 0.5202, -0.0165, -0.0073],
        [ 0.4740, -0.0109, -0.0035],
        [ 0.4892, -0.0097,  0.0159]])


2022-06-15 14:44:20.510300: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-06-15 14:44:20.516991: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1996445000 Hz


tensor([[-3.1186],
        [-1.8932],
        [-3.7288],
        [-0.7036],
        [-1.3467]], dtype=torch.float64)


## Bayesian Optimization

In [6]:
n_steps = 30
best_y = torch.max(train_y)
phys = False

for i in range(n_steps):  
    
    if (phys == True):
        normed_train_x = transformer_x.forward(train_x) 
        normed_train_y = transformer_y.forward(train_y)[0]
        
        x_new, model = get_BO_point(normed_train_x, normed_train_y, 
                                    bounds=normed_bounds, 
                                    phys=phys)
        
        train_x = torch.cat((train_x, transformer_x.untransform(x_new)))
        new_y = torch.tensor(evaluate(train_x[-1])).reshape(1,1)
        train_y = torch.cat((train_y, new_y))
    else:
        x_new, model = get_BO_point(train_x, train_y, 
                                    bounds=bounds.transpose(0,1), 
                                    input_transform=transformer_x, 
                                    outcome_transform=transformer_y, 
                                    phys=phys)
        
        train_x = torch.cat((train_x, x_new))
        new_y = torch.tensor(evaluate(train_x[-1])).reshape(1,1)
        train_y = torch.cat((train_y, new_y))
    
    if (new_y > best_y):
        best_y = new_y
        color = '\033[95m', '\033[0m'
    else: 
        color = '\u001b[30m', '\033[0m'
    
    print("iteration        target         SOL          SQ          CQ")
    print(f'{color[0]}{i+1}              {train_y[-1,0]:.5f}     {train_x[-1,0]:.5f}     {train_x[-1,1]:.5f}     {train_x[-1,2]:.5f}{color[1]}')

iteration        target         SOL          SQ          CQ
1              -1.44527     0.45488     -0.01747     0.00476
iteration        target         SOL          SQ          CQ
2              -1.47729     0.45959     -0.00442     -0.01281
iteration        target         SOL          SQ          CQ
3              -0.58053     0.47538     -0.00289     0.00331
iteration        target         SOL          SQ          CQ
4              -0.60954     0.48215     0.00289     -0.00445
iteration        target         SOL          SQ          CQ
5              -0.92636     0.48310     0.01421     0.00571
iteration        target         SOL          SQ          CQ
6              -0.62279     0.47399     0.00888     -0.00440
iteration        target         SOL          SQ          CQ
7              -0.71795     0.48084     0.02000     -0.01381
iteration        target         SOL          SQ          CQ
8              -0.85908     0.47930     0.00500     -0.02000
iteration        target         

In [7]:
"""
#define acquisition function
from botorch.acquisition.analytic import UpperConfidenceBound, ExpectedImprovement
from botorch.optim import optimize_acqf

#optimize
n_steps = 45
for i in range(n_steps):
    #best_normed_y = torch.max(normed_train_y)
    UCB = UpperConfidenceBound(gp, beta=2.5)
    #EI = ExpectedImprovement(gp, best_normed_y)

    bounds = torch.cat((torch.zeros(1,3), torch.ones(1,3)), 0)
    candidate, acq_value = optimize_acqf(UCB, bounds = bounds, num_restarts = 20, q = 1, raw_samples = 20)

    train_x = torch.cat((train_x, transformer_x.backward(candidate)))
    normed_train_x = transformer_x.forward(train_x)

    new_y = torch.tensor(evaluate(train_x[-1][0], train_x[-1][1], train_x[-1][2])).reshape(1,1)
    train_y = torch.cat((train_y, new_y))
    
    print("iteration        target         varx         vary         varz")
    print(f'{i+1}              {train_y[-1][0]:.5f}      {train_x[-1][0]:.5f}      {train_x[-1][1]:.5f}      {train_x[-1][2]:.5f}')
    print(torch.max(train_y))
    
    transformer_y = transformer.Transformer(train_y, 'standardize')
    normed_train_y = transformer_y.forward(train_y)

    gp = SingleTaskGP(normed_train_x, normed_train_y)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_model(mll);
"""

'\n#define acquisition function\nfrom botorch.acquisition.analytic import UpperConfidenceBound, ExpectedImprovement\nfrom botorch.optim import optimize_acqf\n\n#optimize\nn_steps = 45\nfor i in range(n_steps):\n    #best_normed_y = torch.max(normed_train_y)\n    UCB = UpperConfidenceBound(gp, beta=2.5)\n    #EI = ExpectedImprovement(gp, best_normed_y)\n\n    bounds = torch.cat((torch.zeros(1,3), torch.ones(1,3)), 0)\n    candidate, acq_value = optimize_acqf(UCB, bounds = bounds, num_restarts = 20, q = 1, raw_samples = 20)\n\n    train_x = torch.cat((train_x, transformer_x.backward(candidate)))\n    normed_train_x = transformer_x.forward(train_x)\n\n    new_y = torch.tensor(evaluate(train_x[-1][0], train_x[-1][1], train_x[-1][2])).reshape(1,1)\n    train_y = torch.cat((train_y, new_y))\n    \n    print("iteration        target         varx         vary         varz")\n    print(f\'{i+1}              {train_y[-1][0]:.5f}      {train_x[-1][0]:.5f}      {train_x[-1][1]:.5f}      {train_x[-